In [8]:
%load_ext autoreload
%autoreload 2

import os
import numpy as np
import pandas as pd

if 'cachai' not in os.listdir('.'):
    os.chdir('../')
    
import cachai.utils.constants as C
import cachai.utils.types as T
from cachai.utils.metrics import evaluate, evaluate_groupby

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
from sklearn.metrics import mean_absolute_error


class TTLSimulator:

    def __init__(self, iterations=1_000):
        self._iterations = iterations
        self._target_params = [
            (50, 5),
            (200, 10),
            (400, 30),
        ]
        # means = np.linspace(10, 500, 10).astype(int)
        # std = np.arange(1, len(means) + 1)**2
        # self._target_params = np.array([means, std]).T

    def update_target_params(self, progress):
        target_params = []
        for param in self._target_params:
            mean = float(
                round(param[0]*np.sin(progress*2*np.pi/2)/(param[0]/2) + param[0], 2)
            )
            std = param[1]
            target_params.append((mean, std))
        self._target_params = target_params

    def feedback(self, y_true, y_pred):
        observation_time = int(min(y_true, y_pred)[0])
        hits = max(0, observation_time-1)
        observation_type = None
        if y_pred[0] < y_true[0]:
            observation_type = C.MISS
        elif y_pred[0] > y_true[0]:
            observation_type = C.STALE
        else:
            observation_type = C.VALID_TTL
        return observation_time, observation_type, hits

    def generate(self):
        # get target
        target_param_index = np.random.randint(0, len(self._target_params))
        target_params = self._target_params[target_param_index]
        y = np.random.normal(target_params[0], target_params[1], 1)

        # generate features from target
        num_features = 1
        correlation = 0.8
        cov_matrix = np.eye(num_features) * (1 - correlation) + np.ones((num_features, num_features)) * correlation
        features = np.random.multivariate_normal(np.ones(num_features) * y, cov_matrix)
        X = features.reshape(1, -1)
        # X = np.full((1, num_features), fill_value=target)
        return X, y

In [10]:
class Experiment():

    DF_COLUMNS = [
        C.EXPERIMENT_NAME, C.MODEL_NAME, C.ITERATION, C.OBSERVATION_TYPE, C.OBSERVATION_TIME,
        C.Y_PRED, C.Y_TRUE, C.HITS, C.MAE
    ]

    def __init__(self, simulator, iterations=1_000):
        self._simulator = simulator
        self._iterations = iterations

    def run(self, experiments):
        df = []
        for experiment in experiments:
            experiment_name = experiment[C.EXPERIMENT_NAME]
            model = experiment['model']
            for i in range(self._iterations):
                X, y_true = self._simulator.generate()
                y_pred = model.predict(X)
                observation_time, observation_type, hits = self._simulator.feedback(y_true, y_pred)
                mae = mean_absolute_error(y_true, y_pred)
                model.observe(observation_time, observation_type, hits, y_pred)
                df.append([
                    experiment_name, model.NAME, i, observation_type, observation_time,
                    y_pred[0], y_true[0], hits, mae
                ])
        return pd.DataFrame(df, columns=Experiment.DF_COLUMNS)

In [11]:
from abc import ABC, abstractmethod


class BaseModel(ABC):

    @property
    @abstractmethod
    def NAME(self) -> str:
        pass

    @abstractmethod
    def predict(self, X: np.array) -> np.array:
        pass

    @abstractmethod
    def observe(
        self,
        observation_time: int,
        observation_type: T.ObservationType,
        hits: int,
        prev_prediction: float
    ) -> None:
        pass

In [12]:
class Model(BaseModel):
    NAME = 'OPTIMAL'

    def __init__(self):
        pass

    def predict(self, X):
        output = X.mean()
        return np.array([output])

    def observe(self, observation_time, observation_type, hits, prev_prediction):
        a = 2+2

In [13]:
simulator = TTLSimulator()
experiment = Experiment(simulator=simulator)
df = experiment.run([
    {
        C.EXPERIMENT_NAME: 'TEST 1',
        'model': Model(),
    },
    {
        C.EXPERIMENT_NAME: 'TEST 2',
        'model': Model(),
    }
])
df.head(10)

,experiment_name,model_name,iteration,observation_type,observation_time,y_pred,y_true,hits,mae
0,TEST 1,OPTIMAL,0,miss,207,207.759566,207.941247,206,0.181680
1,TEST 1,OPTIMAL,1,stale,50,51.334882,50.905924,49,0.428958
2,TEST 1,OPTIMAL,2,miss,202,202.832349,203.218984,201,0.386635
3,TEST 1,OPTIMAL,3,miss,48,48.775985,48.914726,47,0.138741
4,TEST 1,OPTIMAL,4,miss,40,40.716509,41.056478,39,0.339969
5,TEST 1,OPTIMAL,5,miss,45,45.510986,45.989989,44,0.479004
6,TEST 1,OPTIMAL,6,stale,46,47.178593,46.600805,45,0.577788
7,TEST 1,OPTIMAL,7,miss,189,189.805831,191.126872,188,1.321040
8,TEST 1,OPTIMAL,8,stale,53,54.903279,53.673474,52,1.229805
9,TEST 1,OPTIMAL,9,miss,46,46.515955,48.217101,45,1.701145


In [14]:
evaluate(df)

,rmse,mae,mabe,msbe,hit_rate,miss_rate,precision,accuracy,recall,false_positive_rate,f1_score,hits_total,hits_mean,len
0,1.012462,0.812565,0.812565,1.025079,0.5035,0.4965,0.0,0.0,0.0,1.0,0,427199,213.5995,2000


In [15]:
evaluate_groupby(df)

,,,rmse,mae,mabe,msbe,hit_rate,miss_rate,precision,accuracy,recall,false_positive_rate,f1_score,hits_total,hits_mean,len
experiment_name,model_name,,,,,,,,,,,,,,,
TEST 1,OPTIMAL,0,1.014952,0.817559,0.817559,1.030127,0.505,0.495,0.0,0.0,0.0,1.0,0,213468,213.468,1000
TEST 2,OPTIMAL,0,1.009966,0.807572,0.807572,1.020031,0.502,0.498,0.0,0.0,0.0,1.0,0,213731,213.731,1000
